# MLP Exercise - fantasy name generation

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
from src.layers import Linear, BatchNorm1d, Tanh
%matplotlib inline


In [2]:
# Get the datasets

import kagglehub

# Download latest version
path = kagglehub.dataset_download("isaacbenge/fantasy-for-markov-generator")

print("Path to dataset files:", path)

d:\dev\namegens\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Fern_PC II\.cache\kagglehub\datasets\isaacbenge\fantasy-for-markov-generator\versions\1


In [3]:
# clean data, normalize strings 
import glob
import pandas as pd 
allcsvs = []
for f in glob.glob(path+"/*.csv") + ["./names_clean.csv"]:
    df = pd.read_csv(f, header=None)
    series = (df[0].str.normalize('NFKD')
                   .str.encode('ascii', errors='ignore')
                   .str.decode('utf-8')
                   .str.lower()
                   .str.replace(r'[^\w\s]', '')
                   .str.replace('/', '')
                   .str.replace('(', '')
                   .str.replace(')', '')
                   .str.strip()
                   
                   )

    allcsvs.append(series)

# Adding names from names_clean.csv
words = pd.concat(allcsvs).to_list()


In [4]:
print(len(words))
words[:8]

28049


['mai', 'ba', 'binh', 'chi', 'cong', 'cuong', 'ha', 'hien']

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: ' ', 2: "'", 3: '-', 4: '_', 5: 'a', 6: 'b', 7: 'c', 8: 'd', 9: 'e', 10: 'f', 11: 'g', 12: 'h', 13: 'i', 14: 'j', 15: 'k', 16: 'l', 17: 'm', 18: 'n', 19: 'o', 20: 'p', 21: 'q', 22: 'r', 23: 's', 24: 't', 25: 'u', 26: 'v', 27: 'w', 28: 'x', 29: 'y', 30: 'z', 0: '.'}
31


In [6]:
# build the dataset
block_size = 8 # context length: how many characters do we take to predict the next one?
g = torch.Generator().manual_seed(2147483647) # for reproducibility

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%


torch.Size([177783, 8]) torch.Size([177783])
torch.Size([21870, 8]) torch.Size([21870])
torch.Size([22348, 8]) torch.Size([22348])


In [7]:
n_embd = 16 # the dimensionality of the character embedding vectors
n_hidden = 128 # the number of neurons in the hidden layer of the MLP
g = torch.Generator().manual_seed(2147483647) # for reproducibility

# Self-Attention layer class
class SelfAttention:
  def __init__(self, n_embd, head_size, block_size, dropout=0.0):
    self.n_embd = n_embd
    self.head_size = head_size
    self.block_size = block_size
    # Create Q, K, V projection matrices
    self.key = Linear(n_embd, head_size, bias=False)
    self.query = Linear(n_embd, head_size, bias=False)
    self.value = Linear(n_embd, head_size, bias=False)
    # Output projection
    self.proj = Linear(head_size, n_embd, bias=False)
    # Causal mask: lower triangular matrix to prevent looking at future tokens
    self.register_buffer = lambda name, val: setattr(self, name, val)
    tril = torch.tril(torch.ones(block_size, block_size))
    self.register_buffer('mask', tril)
    
  def __call__(self, x):
    # x shape: (batch_size, block_size, n_embd)
    B, T, C = x.shape
    
    # Compute Q, K, V
    k = self.key(x)   # (B, T, head_size)
    q = self.query(x) # (B, T, head_size)
    v = self.value(x) # (B, T, head_size)
    
    # Compute attention scores: (B, T, head_size) @ (B, head_size, T) -> (B, T, T)
    wei = q @ k.transpose(-2, -1) * self.head_size**-0.5  # scaled dot-product
    wei = wei.masked_fill(self.mask[:T, :T] == 0, float('-inf'))  # apply causal mask
    wei = F.softmax(wei, dim=-1)  # (B, T, T)
    
    # Apply attention to values
    out = wei @ v  # (B, T, head_size)
    
    # Output projection
    out = self.proj(out)  # (B, T, n_embd)
    
    self.out = out
    return out
  
  def parameters(self):
    return (self.key.parameters() + self.query.parameters() + 
            self.value.parameters() + self.proj.parameters())

C = torch.randn((vocab_size, n_embd),            generator=g)
# Attention layer (single head with head_size = n_embd)
head_size = n_embd  # Use same size as embedding dimension
attention = SelfAttention(n_embd, head_size, block_size)

layers = [
  Linear(n_embd * block_size, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size),
]
with torch.no_grad():
  # last layer: make less confident
  layers[-1].gamma *= 0.1
  #layers[-1].weight *= 0.1
  # all other layers: apply gain
  for layer in layers[:-1]:
    if isinstance(layer, Linear):
      layer.weight *= 1.0 #5/3

parameters = [C] + attention.parameters() + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

122030


In [ ]:
# same optimization as last time
max_steps = 200000
batch_size = 64
lossi = []
ud = []

for i in range(max_steps):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
  
  # forward pass
  emb = C[Xb] # embed the characters into vectors: (batch_size, block_size, n_embd)
  
  # Apply attention layer (keeps sequence dimension)
  x = attention(emb)  # (batch_size, block_size, n_embd)
  
  # Flatten for MLP layers
  x = x.view(emb.shape[0], -1) # concatenate the vectors: (batch_size, block_size * n_embd)
  
  # Pass through MLP layers
  for layer in layers:
    x = layer(x)
  loss = F.cross_entropy(x, Yb) # loss function
  
  # backward pass
  attention.out.retain_grad()  # Store attention output gradient
  for layer in layers:
    layer.out.retain_grad() # AFTER_DEBUG: would take out retain_graph
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  lr = 0.1 if i < 150000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())
  with torch.no_grad():
    ud.append([((lr*p.grad).std() / p.data.std()).log10().item() for p in parameters])

  #if i >= 1000:
  #  break # AFTER_DEBUG: would take out obviously to run full optimization

      0/ 200000: 3.4396
  10000/ 200000: 2.5552


In [ ]:

#plt.plot(lossi)
plt.plot(torch.tensor(lossi).view(-1, 1000).mean(1))

In [ ]:
# visualize histograms
plt.figure(figsize=(20, 4)) # width and height of the plot
legends = []
for i, layer in enumerate(layers[:-1]): # note: exclude the output layer
  if isinstance(layer, Tanh):
    t = layer.out
    print('layer %d (%10s): mean %+.2f, std %.2f, saturated: %.2f%%' % (i, layer.__class__.__name__, t.mean(), t.std(), (t.abs() > 0.97).float().mean()*100))
    hy, hx = torch.histogram(t, density=True)
    plt.plot(hx[:-1].detach(), hy.detach())
    legends.append(f'layer {i} ({layer.__class__.__name__}')
plt.legend(legends);
plt.title('activation distribution')

In [ ]:
# visualize histograms
plt.figure(figsize=(20, 4)) # width and height of the plot
legends = []
for i, layer in enumerate(layers[:-1]): # note: exclude the output layer
  if isinstance(layer, Tanh):
    t = layer.out.grad
    print('layer %d (%10s): mean %+f, std %e' % (i, layer.__class__.__name__, t.mean(), t.std()))
    hy, hx = torch.histogram(t, density=True)
    plt.plot(hx[:-1].detach(), hy.detach())
    legends.append(f'layer {i} ({layer.__class__.__name__}')
plt.legend(legends);
plt.title('gradient distribution')

In [ ]:
# visualize histograms
plt.figure(figsize=(20, 4)) # width and height of the plot
legends = []
for i,p in enumerate(parameters):
  t = p.grad
  if p.ndim == 2:
    print('weight %10s | mean %+f | std %e | grad:data ratio %e' % (tuple(p.shape), t.mean(), t.std(), t.std() / p.std()))
    hy, hx = torch.histogram(t, density=True)
    plt.plot(hx[:-1].detach(), hy.detach())
    legends.append(f'{i} {tuple(p.shape)}')
plt.legend(legends)
plt.title('weights gradient distribution');

In [ ]:
plt.figure(figsize=(20, 4))
legends = []
for i,p in enumerate(parameters):
  if p.ndim == 2:
    plt.plot([ud[j][i] for j in range(len(ud))])
    legends.append('param %d' % i)
plt.plot([0, len(ud)], [-3, -3], 'k') # these ratios should be ~1e-3, indicate on plot
plt.legend(legends);


In [ ]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  
  # Apply attention layer
  x = attention(emb)  # (N, block_size, n_embd)
  
  # Flatten for MLP layers
  x = x.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  for layer in layers:
    x = layer(x)
  loss = F.cross_entropy(x, y)
  print(split, loss.item())

# put layers into eval mode
for layer in layers:
  layer.training = False
split_loss('train')
split_loss('val')

In [ ]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 22)

for _ in range(25):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass the neural net
      emb = C[torch.tensor([context])] # (1,block_size,n_embd)
      
      # Apply attention layer
      x = attention(emb)  # (1, block_size, n_embd)
      
      # Flatten for MLP layers
      x = x.view(emb.shape[0], -1) # concatenate the vectors
      for layer in layers:
        x = layer(x)
      logits = x
      probs = F.softmax(logits, dim=1)
      # sample from the distribution
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      # shift the context window and track the samples
      context = context[1:] + [ix]
      out.append(ix)
      # if we sample the special '.' token, break
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out)) # decode and print the generated word

In [ ]:
# sample from the model forcing more words
g = torch.Generator().manual_seed(2147483647 + 69)

for _ in range(25):
    has_space = False
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass the neural net
      emb = C[torch.tensor([context])] # (1,block_size,n_embd)
      
      # Apply attention layer
      x = attention(emb)  # (1, block_size, n_embd)
      
      # Flatten for MLP layers
      x = x.view(emb.shape[0], -1) # concatenate the vectors
      for layer in layers:
        x = layer(x)
      logits = x
      probs = F.softmax(logits, dim=1)
      # sample from the distribution
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      # shift the context window and track the samples
      # if we sample the special '.' token, replace with space if no space, otherwise break
      if itos[ix] == ' ':
        has_space = True
      if ix == 0:
        if has_space==False:
            ix = stoi[' ']
            has_space = True
        else:
            break
      context = context[1:] + [ix]
      out.append(ix)

    
    print(''.join(itos[i] for i in out)) # decode and print the generated word

In [ ]:
# sample from the model forcing "the"
g = torch.Generator().manual_seed(2147483647 + 67)

for _ in range(25):
    has_the = False
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass the neural net
      emb = C[torch.tensor([context])] # (1,block_size,n_embd)
      
      # Apply attention layer
      x = attention(emb)  # (1, block_size, n_embd)
      
      # Flatten for MLP layers
      x = x.view(emb.shape[0], -1) # concatenate the vectors
      for layer in layers:
        x = layer(x)
      logits = x
      probs = F.softmax(logits, dim=1)
      # sample from the distribution
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      # shift the context window and track the samples
      # if we sample the special '.' token, replace with space if no space, otherwise break
      if ix == 0:
        if has_the==False:
            context = context[5:] + [stoi[' '], stoi['t'], stoi['h'], stoi['e'], stoi[' ']]
            has_the = True
            out = out[5:] + [stoi[' '], stoi['t'], stoi['h'], stoi['e'], stoi[' ']]
        else:
            break
      else:
        context = context[1:] + [ix]
        out.append(ix)

    
    print(''.join(itos[i] for i in out)) # decode and print the generated word

In [ ]:
print(out)
out[4:] + [stoi['t'], stoi['h'], stoi['e'], stoi[' ']]

In [ ]:
stoi

In [ ]:
C.shape

### from andrejs
train 2.0193495750427246
val 2.156538963317871
### bigger network
train 1.9244529008865356
val 2.1407828330993652
### adjusted sizes + more data
train 1.642111897468567
val 2.2296860218048096

good names tho